In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

ee.Initialize()

### Imports and Constants

In [136]:
import ee
import geemap

Map = geemap.Map()

# Emily's Areas
region1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]])
region2 = ee.Geometry.Polygon(
        [[[25.85, -10.6],
          [25.85, -10.7],
          [25.95, -10.7],
          [25.95, -10.6]]])
region3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]])
region4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]])
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]])
total = ee.Geometry.Polygon(
        [[[25.8, -10.5],
          [25.8, -10.8],
          [26.1, -10.8],
          [26.1, -10.5]]])
# Rishi Area
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])
# Ray Areas 
roi1 = ee.Geometry.Polygon(
        [[[29.55, 3.16],
          [29.55, 3.1],
          [29.63, 3.1],
          [29.63, 3.16]]])
roi2 = ee.Geometry.Polygon(
        [[[29.76, 3.16],
          [29.76, 3.12],
          [29.8, 3.12],
          [29.8, 3.16]]])
roi3 = ee.Geometry.Polygon(
        [[[30.25, 1.8],
          [30.25, 1.7],
          [30.35, 1.7],
          [30.35, 1.8]]])

# Modis Data and Visualization
modis = ee.ImageCollection("MODIS/006/MCD43A4")
TrueColorVis = {"min":0,"max":4000,"gamma":1.4}

# Sentinel-2 Data and Visualization
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
rgbVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

### Classifier

In [137]:
# Training Data - used on Emily Region1, Modis, 2020 Composite
nonVeg2020 = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([25.884051513674073, -10.612839387311823]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([25.809550476076417, -10.77881944655002]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([25.86070556640845, -10.781517557036542]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([25.87787170410376, -10.751837012248982]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([25.9788085937522, -10.64185870124958]),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([26.00732993260123, -10.631115566883805]),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([26.06500815525748, -10.63246528279504]),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([26.01831626072623, -10.523794114923895]),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Point([25.932501220912915, -10.613290891172372]),
            {
              "landcover": 1,
              "system:index": "8"
            })])
veg2020 = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([25.971598815920167, -10.63713481487796]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([25.828776550295167, -10.650968846848661]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([26.023783874513917, -10.737332956336642]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([25.8689453125022, -10.767689485172806]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([25.88439483642798, -10.790286248993706]),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([25.814382544905918, -10.56429718435804]),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([26.063634864241855, -10.602770155202851]),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([26.077367774398105, -10.745820038885466]),
            {
              "landcover": 2,
              "system:index": "7"
            })])

# Merge training data
training = veg2020.merge(nonVeg2020)

# Image for Training Data
trainingImage = modis \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(total)) \
  .select(['Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band4',
  'Nadir_Reflectance_Band3']) \
  .median().clip(total)

# Overlay the point on the image to get training data.
training = trainingImage.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 10
})

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': trainingImage.bandNames()
})

### Select Desired Region

In [138]:
mainRegion = region3

### Create Composite Images (MODIS)

In [139]:
rgb2000 = modis \
  .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
  .filter(ee.Filter.bounds(mainRegion)) \
  .select(['Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band4',
  'Nadir_Reflectance_Band3']) \
  .median().clip(mainRegion)
rgb2020 = modis \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(mainRegion)) \
  .select(['Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band4',
  'Nadir_Reflectance_Band3']) \
  .median().clip(mainRegion)

### Create .01x.01 Regions

In [140]:
# must be rectangular geometry
def divide_geometry(geo):
    regions = []
    coords = geo.coordinates().getInfo()[0]
    i = coords[0][0]
    while i < coords[1][0]:
        j = coords[2][1]
        while j > coords[0][1]:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.025],
                  [i+.025, j-.025],
                  [i+.025, j]]])
            regions.append((region))
            j = j-.025
        i = i+.025
    return regions

In [141]:
regions = divide_geometry(mainRegion)
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 110


### Percent Vegetation Loss

In [142]:
def vegLoss(regionList): 
    Map.centerObject(mainRegion, 12)
    Map.addLayer(rgb2000, TrueColorVis, '2000', False)
    Map.addLayer(rgb2020, TrueColorVis, '2020', False)
    classified1 = rgb2000.classify(classifier)
    classified2 = rgb2020.classify(classifier)
    Map.addLayer(classified1, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2000_classified', False)
    Map.addLayer(classified2, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2020_classified', False)
    whole = s2 \
            .filter(ee.Filter.bounds(mainRegion)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(mainRegion)
    Map.addLayer(whole, rgbVis, 'whole', False)
    passed = []
    num = 0
    for region in regionList:
        image2000 = modis \
            .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
        image2020 = modis \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.bounds(region)) \
            .select(['Nadir_Reflectance_Band1',
                    'Nadir_Reflectance_Band4',
                    'Nadir_Reflectance_Band3']) \
            .median().clip(region)
        # Classify the images
        classified2020 = image2020.classify(classifier)
        # Map.addLayer(classified2020, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, str(num) + ' 2020_classified', False)

        classified2000 = image2000.classify(classifier)
        # Map.addLayer(classified2000, {'min': 1, 'max': 2, 'palette': ['red', 'green']},  str(num) + ' 2000_classified', False)

        veg1 = classified2000.eq(2)
        veg1Image = veg1.multiply(ee.Image.pixelArea())
        area1 = veg1Image.reduceRegion(**{
          'reducer': ee.Reducer.sum(),
          'geometry': region,
          'scale': 500,
          'maxPixels': 1e10
          })
        veg1SqKm = ee.Number(area1.get('classification')).divide(1e6).round().getInfo()
        #print(str(num) + ' Area of Vegetation in 2000: ', veg1SqKm)
        
        veg2 = classified2020.eq(2)
        veg2Image = veg2.multiply(ee.Image.pixelArea())
        area2 = veg2Image.reduceRegion(**{
          'reducer': ee.Reducer.sum(),
          'geometry': region,
          'scale': 500,
          'maxPixels': 1e10
          })
        veg2SqKm = ee.Number(area2.get('classification')).divide(1e6).round().getInfo()
        #print(str(num) + ' Area of Vegetation in 2020: ', veg2SqKm)
        num = num+1
        if (veg1SqKm == 0) and (veg2SqKm == 0):
            passed.append(region)
            print(str(num) + ': No Vegetation at Start or End')
        elif veg1SqKm > 0:
            #percent decrease of vegetation area = ((original - new) / (original)) * 100
            percentDecrease = ((veg1SqKm-veg2SqKm)/veg1SqKm)*100
            print(str(num) + ': ' + str(percentDecrease) + '% Vegetation Loss')
            if percentDecrease >= 25:
                passed.append(region)
        else:
            print(str(num) + ': Vegetation Increased from 0 Initially')
    if len(passed) > 0:
        layer = ee.Geometry.MultiPolygon(passed)
        image = s2 \
            .filter(ee.Filter.bounds(layer)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer)
        Map.addLayer(image, rgbVis, 'pass')
    Map
    return passed    

In [143]:
passed_veg = vegLoss(regions)

1: 0.0% Vegetation Loss
2: 85.71428571428571% Vegetation Loss
3: 71.42857142857143% Vegetation Loss
4: 14.285714285714285% Vegetation Loss
5: 50.0% Vegetation Loss
6: 100.0% Vegetation Loss
7: 100.0% Vegetation Loss
8: 37.5% Vegetation Loss
9: 0.0% Vegetation Loss
10: 0.0% Vegetation Loss
11: -16.666666666666664% Vegetation Loss
12: -16.666666666666664% Vegetation Loss
13: 14.285714285714285% Vegetation Loss
14: 66.66666666666666% Vegetation Loss
15: 75.0% Vegetation Loss
16: 100.0% Vegetation Loss
17: 100.0% Vegetation Loss
18: 12.5% Vegetation Loss
19: 0.0% Vegetation Loss
20: 0.0% Vegetation Loss
21: 0.0% Vegetation Loss
22: 28.57142857142857% Vegetation Loss
23: 28.57142857142857% Vegetation Loss
24: 87.5% Vegetation Loss
25: 100.0% Vegetation Loss
26: 100.0% Vegetation Loss
27: 50.0% Vegetation Loss
28: 25.0% Vegetation Loss
29: 0.0% Vegetation Loss
30: -16.666666666666664% Vegetation Loss
31: 25.0% Vegetation Loss
32: 37.5% Vegetation Loss
33: 85.71428571428571% Vegetation Loss
3

In [144]:
# next:
# compare level with rishi and ray -> what size square should we use?
## right now i have .025x.025 -> edit it to long and lat like in Ray's code
# get detailed justification for the uses of these levels/ percentage of vegetation loss -> could do research on mine size vs square size????
# look into what Eamon said - I am confused and bad at exporting so hopefully since I figured out the stuff above R/R can figure out that??

## nir swir1????

### NDMI

In [145]:
def create_ndmi_layers_modis(start, end, geometry):
    images = []
    ndmiStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiStart2 = ndmiStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiStart2, start))
    ndmiEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    ndmiEnd2 = ndmiEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
    images.append((ndmiEnd2, end))
    return images

In [146]:
def extract_ndmi_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDMI': stats.get('NDMI')
    }
    
    return ee.Feature(None, properties)

In [147]:
def ndmiLoss(regionList):
    passed_ndmi = []
    num = 1
    for region in regionList:
        imagesNDMI = ee.ImageCollection(create_ndmi_layers_modis(2000, 2020, region))
        dataNDMI = ee.FeatureCollection(imagesNDMI.map(extract_ndmi_modis))
        dfNDMI = geemap.ee_to_pandas(dataNDMI)
        loss = dfNDMI.iat[0,1]-dfNDMI.iat[1,1]
        print(str(num) + ': ' + str(loss) + ' NDMI Loss')
        num = num+1
        if loss > 0.025:
             passed_ndmi.append(region)
    if len(passed_ndmi) > 0:
        layer_ndmi = ee.Geometry.MultiPolygon(passed_ndmi)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ndmi)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ndmi)
        Map.addLayer(image, rgbVis, 'pass ndmi')
    Map
    return passed_ndmi

In [148]:
passed_ndmi = ndmiLoss(passed_veg)

1: 0.013207887379938086 NDMI Loss
2: 0.023268020955814813 NDMI Loss
3: 0.06026568668563541 NDMI Loss
4: 0.09843195047737925 NDMI Loss
5: 0.08492298595175606 NDMI Loss
6: 0.03809464349999159 NDMI Loss
7: -0.021120707271541774 NDMI Loss
8: 0.01946323647436546 NDMI Loss
9: 0.08889886701728211 NDMI Loss
10: 0.03862408640224135 NDMI Loss
11: -0.03808729201120729 NDMI Loss
12: -0.019803534127046 NDMI Loss
13: -0.013587421883557663 NDMI Loss
14: 0.0154045882902982 NDMI Loss
15: 0.09860713033116345 NDMI Loss
16: -0.012521851280353918 NDMI Loss
17: -0.006317314925925202 NDMI Loss
18: 0.012950485907046588 NDMI Loss
19: -0.03576928001339323 NDMI Loss
20: -0.025381233230019198 NDMI Loss
21: -0.010903446346871353 NDMI Loss
22: 0.03522984927129147 NDMI Loss
23: 0.02969013150251469 NDMI Loss
24: -0.00039379031012681505 NDMI Loss
25: -0.027538248696625883 NDMI Loss
26: 0.032227344785843645 NDMI Loss
27: -0.012818750994604177 NDMI Loss
28: -0.01739895299431653 NDMI Loss
29: 0.03765947769508472 NDMI Los

### NIR/SWIR2

In [149]:
def create_nirSwir2_layers_modis(start, end, geometry):
    images = []
    nirSwir2Start = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2Start2 = nirSwir2Start.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2Start2, start))
    nirSwir2End = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirSwir2End2 = nirSwir2End.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band7']).rename('nirSwir2')
    images.append((nirSwir2End2, end))
    return images

In [150]:
def extract_nirSwir2_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirSwir2': stats.get('nirSwir2')
    }
    
    return ee.Feature(None, properties)

In [151]:
def ns2Loss(regionList):
    passed_ns2 = []
    num = 1
    for region in regionList:
        imagesNirSwir2 = ee.ImageCollection(create_nirSwir2_layers_modis(2000, 2020, region))
        dataNirSwir2 = ee.FeatureCollection(imagesNirSwir2.map(extract_nirSwir2_modis))
        dfNirSwir2 = geemap.ee_to_pandas(dataNirSwir2)
        loss = dfNirSwir2.iat[0,1]-dfNirSwir2.iat[1,1]
        print(str(num) + ': ' + str(loss) + ' NIR/SWIR2 Loss')
        num = num+1
        if loss > 0:
            passed_ns2.append(region)
    if len(passed_ns2) > 0:
        layer_ns2 = ee.Geometry.MultiPolygon(passed_ns2)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ns2)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ns2)
        Map.addLayer(image, rgbVis, 'pass nir/swir2')
    Map
    return passed_ns2

In [152]:
passed_ns2 = ns2Loss(passed_ndmi)

1: 0.0418200906055573 NIR/SWIR2 Loss
2: 0.15382574958365405 NIR/SWIR2 Loss
3: 0.15533242329741848 NIR/SWIR2 Loss
4: -0.0006622869089109584 NIR/SWIR2 Loss
5: 0.122432725969844 NIR/SWIR2 Loss
6: 0.07071089634540431 NIR/SWIR2 Loss
7: 0.18875850388945198 NIR/SWIR2 Loss
8: 0.11113431792855158 NIR/SWIR2 Loss
9: 0.07884905084986385 NIR/SWIR2 Loss
10: 0.013868225481236951 NIR/SWIR2 Loss
11: 0.13769771708820627 NIR/SWIR2 Loss
12: 0.1594448251519511 NIR/SWIR2 Loss
13: 0.1326977211564757 NIR/SWIR2 Loss
14: 0.14708983485667684 NIR/SWIR2 Loss
15: 0.12060959022077353 NIR/SWIR2 Loss
16: 0.22176634514735427 NIR/SWIR2 Loss
17: 0.06561663552570446 NIR/SWIR2 Loss
18: 0.12549165632754222 NIR/SWIR2 Loss
19: 0.09549955343165348 NIR/SWIR2 Loss
20: 0.06782011594969478 NIR/SWIR2 Loss
21: 0.21400691461201826 NIR/SWIR2 Loss
22: 0.22285229632518433 NIR/SWIR2 Loss
23: 0.12342410451413321 NIR/SWIR2 Loss
24: 0.008709591663264682 NIR/SWIR2 Loss
25: 0.06188374430614263 NIR/SWIR2 Loss
26: 0.15782559209301927 NIR/SWIR2 

### NIR/G

In [153]:
def create_nirG_layers_modis(start, end, geometry):
    images = []
    nirGStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirGStart2 = nirGStart.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band4']).rename('nirG')
    images.append((nirGStart2, start))
    nirGEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    nirGEnd2 = nirGEnd.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band4']).rename('nirG')
    images.append((nirGEnd2, end))
    return images

In [154]:
def extract_nirG_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'nirG': stats.get('nirG')
    }
    
    return ee.Feature(None, properties)

In [155]:
def ngLoss(regionList):
    passed_ng = []
    num = 1
    for region in regionList:
        imagesNirG = ee.ImageCollection(create_nirG_layers_modis(2000, 2020, region))
        dataNirG = ee.FeatureCollection(imagesNirG.map(extract_nirG_modis))
        dfNirG = geemap.ee_to_pandas(dataNirG)
        loss = dfNirG.iat[0,0]-dfNirG.iat[1,0]
        print(str(num) + ': ' + str(loss) + ' NIR/G Loss')
        num = num+1
        if loss > 0.01:
            passed_ng.append(region)
    if len(passed_ng) > 0:
        layer_ng = ee.Geometry.MultiPolygon(passed_ng)
        image = s2 \
            .filter(ee.Filter.bounds(layer_ng)) \
            .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select('B.*') \
            .median().clip(layer_ng)
        Map.addLayer(image, rgbVis, 'pass nir/g')
    Map
    return passed_ng

In [156]:
passed_ng = ngLoss(passed_ns2)

1: 0.027393953844284114 NIR/G Loss
2: 0.07560538618414891 NIR/G Loss
3: 0.07497677687901605 NIR/G Loss
4: 0.059956451035801495 NIR/G Loss
5: 0.025951148658803436 NIR/G Loss
6: 0.11302337755243813 NIR/G Loss
7: 0.06236592050345935 NIR/G Loss
8: 0.0452339373077737 NIR/G Loss
9: 0.017537224357029424 NIR/G Loss
10: 0.07164843508980367 NIR/G Loss
11: 0.08667622594943153 NIR/G Loss
12: 0.07947288486822368 NIR/G Loss
13: 0.10152455256008774 NIR/G Loss
14: 0.09980784735975484 NIR/G Loss
15: 0.11304103665777177 NIR/G Loss
16: 0.041981134937547515 NIR/G Loss
17: 0.15852641791549282 NIR/G Loss
18: 0.09086884048182331 NIR/G Loss
19: 0.04827486697696237 NIR/G Loss
20: 0.11083390405605542 NIR/G Loss
21: 0.14852280333545714 NIR/G Loss
22: 0.06275372168024707 NIR/G Loss
23: 0.013158987428561475 NIR/G Loss
24: 0.06215834546364141 NIR/G Loss
25: 0.09206396151956975 NIR/G Loss
26: 0.03216340960602104 NIR/G Loss
27: 0.08072054577614585 NIR/G Loss
28: 0.011563294988384265 NIR/G Loss
29: 0.07360229435261145

In [161]:
Map

Map(center=[-10.62500765669145, 26.12500000000105], controls=(WidgetControl(options=['position', 'transparent_…